# 主成分分析の結果を用いた分類

- 主成分分析の結果（主成分）を用いて分類
- 次元削減が可能かどうかAUCを比較

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

iris = sns.load_dataset('iris')

# 2値データへの分類のため、species から setosa を除外
# speciesを 0, 1 にするためにダミー変数化
iris_dummies = pd.get_dummies(data=iris.query('species!="setosa"'), drop_first=True)
# 確認
iris_dummies.iloc[[0, 50]]

In [ ]:
# 説明変数：species_virginica 列を削除
X = iris_dummies.drop(['species_virginica'], axis=1)

# 標準化
# - 主成分分析では分散（平均値からのばらつき）を用いるため
#   標準化により平均値を 0 に揃える
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)

# 標準化した DataFrame
X_std = pd.DataFrame(sc.transform(X), index=X.index, columns=X.columns)

### 全ての主成分の説明率

In [ ]:
from sklearn.decomposition import PCA

# 主成分分析 PCA の初期化
# - n_components=None: 主成分はもとの説明変数と同数
pca = PCA(n_components=None)

# 学習結果を DataFrame
X_transformed = pd.DataFrame(pca.fit_transform(X_std))

# 結果
# - explained_variance_ratio_: 各主成分が元のデータを説明する割合
print('各次元の説明率: {}'.format(pca.explained_variance_ratio_))
print('累積説明率: {:.3f}'.format(sum(pca.explained_variance_ratio_)))

3値の時とは、若干異なる結果

### ロジスティック回帰による分類

- 主成分の数を変えて分類

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

def Logistic_Model(X, Y):
    """
    ロジスティック回帰モデルを作成
    """
    model = LogisticRegression(C=10000.0, max_iter=10000000)
    model.fit(X, Y)
    return model

def calc_AUC(model, X, Y):
    """
    AUCの算出
    """
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    return auc(fpr, tpr)

- 主成分による分類

In [ ]:
# 目的変数：species_virginica 列　
Y = iris_dummies['species_virginica']

for n in range(1, 5):
    X_df = X_transformed.iloc[:, 0:n]
    model = Logistic_Model(X_df, Y)
    AUC = calc_AUC(model, X_df, Y)
    print('n={}: AUC={:.3f}'.format(n, AUC))

- もとのデータによる分類

In [ ]:
model = Logistic_Model(X, Y)
AUC = calc_AUC(model, X, Y)
print('AUC={:.3f}'.format(AUC))

主成分3つで、もとのデータと同じAUCで分類可能